In [1]:
from cblearn import datasets
import numpy as np
import cblearn
import tqdm
import time
import sys
sys.path.append('../..')
from comparisonHC import HandlerQuadruplets, ListQuadruplets, get_AddS_quadruplets, get_MulK_quadruplets, ComparisonHC

In [2]:
#fetching nature scene dataset
data = datasets.fetch_nature_scene_similarity()

In [3]:
data

{'triplet': array([[  3,  12,  18],
        [ 66, 111, 106],
        [111,  66,  39],
        ...,
        [ 11,   8,  18],
        [ 27, 111,  48],
        [101, 106,  48]], dtype=int64),
 'image_label': array(['art114.jpg', 'bost100.jpg', 'bost101.jpg', 'bost102.jpg',
        'bost103.jpg', 'bost190.jpg', 'bost98.jpg', 'cdmc101.jpg',
        'cdmc12.jpg', 'cdmc271.jpg', 'cdmc277.jpg', 'cdmc278.jpg',
        'cdmc280.jpg', 'cdmc281.jpg', 'cdmc282.jpg', 'cdmc283.jpg',
        'cdmc284.jpg', 'cdmc290.jpg', 'cdmc291.jpg', 'cdmc292.jpg',
        'cdmc293.jpg', 'cdmc306.jpg', 'cdmc313.jpg', 'cdmc315.jpg',
        'cdmc317.jpg', 'cdmc318.jpg', 'cdmc319.jpg', 'cdmc331.jpg',
        'cdmc333.jpg', 'cdmc338.jpg', 'land302.jpg', 'land309.jpg',
        'land334.jpg', 'land339.jpg', 'land340.jpg', 'land341.jpg',
        'land367.jpg', 'land368.jpg', 'land369.jpg', 'land370.jpg',
        'land371.jpg', 'land372.jpg', 'land374.jpg', 'land386.jpg',
        'land480.jpg', 'land606.jpg', 'land616.jpg'

In [4]:
#getting raw triplets from nature scene data
triplets_raw = data['triplet']

In [5]:
n = np.amax(triplets_raw) + 1

In [6]:
#objects in the data
n

120

In [7]:
n_triplets_raw = triplets_raw.shape[0]

In [8]:
#raw triplets in the data(number of responses)
n_triplets_raw

3355

In [9]:
#getting standard triplets from raw triplets. Random triplets are of form (i,j,k) where s_ij>s_ik
triplets = cblearn.preprocessing.triplets_from_oddoneout(data['triplet'])

In [10]:
#getting quadruplets of form (i,j,i,k) from triplets of form (i,j,k)
quadruplets = np.transpose(np.array([triplets[:,0],triplets[:,1],triplets[:,0],triplets[:,2]]))

In [11]:
n_quadruplets = quadruplets.shape[0]

In [12]:
#standard quadruplets obtained from the data
n_quadruplets

6710

In [13]:
Oracle = ListQuadruplets(quadruplets,n,n_quadruplets)

In [14]:
adds_similarities = get_AddS_quadruplets(Oracle,n)

In [15]:
#running comparison hierarchical clustering using AddS-4 similarities
chc = ComparisonHC(adds_similarities,n)
chc.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc.time_elapsed))
print("Revenue with AddS-4: {:.3e}".format(-chc.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 5.58 seconds.
Revenue with AddS-4: 2.650e+05


In [16]:
mulk_similarities = get_MulK_quadruplets(Oracle,n)

In [17]:
#getting 4K-AL similarities
al4k_similarities = mulk_similarities + 2*adds_similarities

In [18]:
#running comparison hierarchical clustering using 4K-AL similarities
chc_al4k = ComparisonHC(al4k_similarities,n)
chc_al4k.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_al4k.time_elapsed))
print("Revenue with 4K-AL: {:.3e}".format(-chc_al4k.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 5.83 seconds.
Revenue with 4K-AL: 2.228e+05
